In [1]:
# importing relevant packages 

import torch 
from torchvision import models
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import glob
import os
import cv2
import matplotlib.pyplot as plt 
from matplotlib import pyplot
import torchvision.transforms as transforms
from torchsummary import summary
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import json
from PIL import Image
import random
import albumentations as A
import glob
from shapely.geometry import Polygon

/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/.Keypoint_detection_notebooks/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
class VGG_model(nn.Module):
    def __init__(self):
        super(VGG_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
            nn.Conv2d(512,512, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512,50, kernel_size=3, padding='same'),
            nn.LeakyReLU(0.1,inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.AdaptiveAvgPool2d(output_size=(8,8))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(3200, 300),
            nn.LeakyReLU(0.1,inplace=True),
            nn.Dropout(0.3),
            nn.Linear(300, 32),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
        

In [3]:
model = VGG_model()
model.load_state_dict(torch.load('./Model_VGG_4_5_6_2000_16300_epoch'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
# model.eval()

In [4]:
def predict_keypoints(img):
    img = cv2.resize(img, (224,224))
    img_tensor = transforms.ToTensor()(img)
    img_mean = img_tensor.mean(dim = (1,2))
    img_std = img_tensor.std(dim = (1,2))
        
    img_normalised = transforms.Normalize(img_mean, img_std)(img_tensor)
    img_normalised = img_normalised.to(device)
    
    key_points = model(img_normalised[None]).flatten().detach().cpu().numpy()
    
    return key_points

In [5]:
def plot_keypoints(img, keypoints):                                                             

    plt.imshow(img)

    x_points = keypoints[0::2]
    y_points = keypoints[1::2]
    plt.scatter(x_points*img.shape[1], y_points*img.shape[0], s = 4, c=(1,0,0))
    plt.show()

In [18]:
def process_num(x):
    return x*random.uniform(0.96, 1.04)

In [35]:
# dataset_names = ['Grasp_dataset_banana', 'Grasp_dataset_chicken', 'Grasp_dataset_egg', 'Grasp_dataset_fig', 'Grasp_dataset_greengrape', 'Grasp_dataset_orange', 'Grasp_dataset_pear', 'Grasp_dataset_pistachio', 'Grasp_dataset_redgrape', 'Grasp_dataset_tomato', 'Grasp_dataset_capsule', 'Grasp_dataset_ibuprofen']
dataset_names = ['Grasp_dataset_test']
header = ['weight_reading_1', 'weight_reading_2', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y', 'label']

current_dir = os. getcwd()

weight_reading_1_arr = []
weight_reading_2_arr = []
pressure_reading_1_arr = []
pressure_reading_2_arr = []
force_reading_1_arr = []
force_reading_2_arr = []

p1_x_arr = []
p1_y_arr = []
p2_x_arr = []
p2_y_arr = []
p3_x_arr = []
p3_y_arr = []
p4_x_arr = []
p4_y_arr = []
p5_x_arr = []
p5_y_arr = []
p6_x_arr = []
p6_y_arr = []
p7_x_arr = []
p7_y_arr = []
p8_x_arr = []
p8_y_arr = []
p9_x_arr = []
p9_y_arr = []
p10_x_arr = []
p10_y_arr = []
p11_x_arr = []
p11_y_arr = []
p12_x_arr = []
p12_y_arr = []
p13_x_arr = []
p13_y_arr = []
p14_x_arr = []
p14_y_arr = []
p15_x_arr = []
p15_y_arr = []
p16_x_arr = []
p16_y_arr = []
label_arr = []


for f in dataset_names:
    
    image_dir = os.path.join(current_dir + '/Grasp_dataset_3', f)
    data_dir = os.path.join(image_dir, f + '.csv')
    df = pd.read_csv(data_dir)
    weight_reading_1_arr = np.concatenate((weight_reading_1_arr, df['weight_reading_1'].values))
    weight_reading_2_arr = np.concatenate((weight_reading_2_arr, df['weight_reading_2'].values))
    pressure_reading_1_arr = np.concatenate((pressure_reading_1_arr, df['pressure_reading_1'].values))
    pressure_reading_2_arr = np.concatenate((pressure_reading_2_arr, df['pressure_reading_2'].values))
    force_reading_1_arr = np.concatenate((force_reading_1_arr, df['force_reading_1'].values))
    force_reading_2_arr = np.concatenate((force_reading_2_arr, df['force_reading_2'].values))
    label_arr = np.concatenate((label_arr, df['label'].values))
    
    j = 0
    
    while j < df.shape[0]:
        image = cv2.imread(os.path.join(image_dir, df.iloc[j]['image_name']))
        keypoints = predict_keypoints(image)
        
#         plot_keypoints(image, keypoints)
        
#         if j == 20:
#             plot_keypoints(image, keypoints)
        

#         plt.imshow(image)

#         plt.scatter(kirigami_x_points*image.shape[1], kirigami_y_points*image.shape[0], s = 4, c=(1,0,0))
#         plt.show()
        
        p1_x_arr.append(keypoints[0])
        p1_y_arr.append(keypoints[1])
        p2_x_arr.append(keypoints[2])
        p2_y_arr.append(keypoints[3])
        p3_x_arr.append(keypoints[4])
        p3_y_arr.append(keypoints[5])
        p4_x_arr.append(keypoints[6])
        p4_y_arr.append(keypoints[7])
        p5_x_arr.append(keypoints[8])
        p5_y_arr.append(keypoints[9])
        p6_x_arr.append(keypoints[10])
        p6_y_arr.append(keypoints[11])
        p7_x_arr.append(keypoints[12])
        p7_y_arr.append(keypoints[13])
        p8_x_arr.append(keypoints[14])
        p8_y_arr.append(keypoints[15])
        p9_x_arr.append(keypoints[16])
        p9_y_arr.append(keypoints[17])
        p10_x_arr.append(keypoints[18])
        p10_y_arr.append(keypoints[19])
        p11_x_arr.append(keypoints[20])
        p11_y_arr.append(keypoints[21])
        p12_x_arr.append(keypoints[22])
        p12_y_arr.append(keypoints[23])
        p13_x_arr.append(keypoints[24])
        p13_y_arr.append(keypoints[25])
        p14_x_arr.append(keypoints[26])
        p14_y_arr.append(keypoints[27])
        p15_x_arr.append(keypoints[28])
        p15_y_arr.append(keypoints[29])
        p16_x_arr.append(keypoints[30])
        p16_y_arr.append(keypoints[31])
        
        j = j + 1
        
#         if j == 2:
#             break
#     break



p1_x_arr = np.array(p1_x_arr) - np.array(p1_x_arr)
p2_x_arr = np.array(p2_x_arr) - np.array(p1_x_arr)
p3_x_arr = np.array(p3_x_arr) - np.array(p1_x_arr)
p4_x_arr = np.array(p4_x_arr) - np.array(p1_x_arr)
p5_x_arr = np.array(p5_x_arr) - np.array(p1_x_arr)
p6_x_arr = np.array(p6_x_arr) - np.array(p1_x_arr)
p7_x_arr = np.array(p7_x_arr) - np.array(p1_x_arr)
p8_x_arr = np.array(p8_x_arr) - np.array(p1_x_arr)
p9_x_arr = np.array(p9_x_arr) - np.array(p1_x_arr)
p10_x_arr = np.array(p10_x_arr) - np.array(p1_x_arr)
p11_x_arr = np.array(p11_x_arr) - np.array(p1_x_arr)
p12_x_arr = np.array(p12_x_arr) - np.array(p1_x_arr)
p13_x_arr = np.array(p13_x_arr) - np.array(p1_x_arr)
p14_x_arr = np.array(p14_x_arr) - np.array(p1_x_arr)
p15_x_arr = np.array(p15_x_arr) - np.array(p1_x_arr)
p16_x_arr = np.array(p16_x_arr) - np.array(p1_x_arr)

p1_y_arr = np.array(p1_y_arr) - np.array(p1_y_arr)
p2_y_arr = np.array(p2_y_arr) - np.array(p1_y_arr)
p3_y_arr = np.array(p3_y_arr) - np.array(p1_y_arr)
p4_y_arr = np.array(p4_y_arr) - np.array(p1_y_arr)
p5_y_arr = np.array(p5_y_arr) - np.array(p1_y_arr)
p6_y_arr = np.array(p6_y_arr) - np.array(p1_y_arr)
p7_y_arr = np.array(p7_y_arr) - np.array(p1_y_arr)
p8_y_arr = np.array(p8_y_arr) - np.array(p1_y_arr)
p9_y_arr = np.array(p9_y_arr) - np.array(p1_y_arr)
p10_y_arr = np.array(p10_y_arr) - np.array(p1_y_arr)
p11_y_arr = np.array(p11_y_arr) - np.array(p1_y_arr)
p12_y_arr = np.array(p12_y_arr) - np.array(p1_y_arr)
p13_y_arr = np.array(p13_y_arr) - np.array(p1_y_arr)
p14_y_arr = np.array(p14_y_arr) - np.array(p1_y_arr)
p15_y_arr = np.array(p15_y_arr) - np.array(p1_y_arr)
p16_y_arr = np.array(p16_y_arr) - np.array(p1_y_arr)


weight_diff_1_arr = np.array(weight_reading_1_arr) - np.array(force_reading_1_arr)
weight_diff_2_arr = np.array(force_reading_2_arr) - np.array(weight_reading_2_arr) 

grasp_dataset = pd.concat([pd.Series(weight_diff_1_arr), pd.Series(weight_diff_2_arr), pd.Series(pressure_reading_1_arr), pd.Series(pressure_reading_2_arr), pd.Series(force_reading_1_arr), pd.Series(force_reading_2_arr), pd.Series(p1_x_arr), pd.Series(p1_y_arr), pd.Series(p2_x_arr), pd.Series(p2_y_arr), pd.Series(p3_x_arr), pd.Series(p3_y_arr), pd.Series(p4_x_arr), pd.Series(p4_y_arr), pd.Series(p5_x_arr), pd.Series(p5_y_arr), pd.Series(p6_x_arr), pd.Series(p6_y_arr), pd.Series(p7_x_arr), pd.Series(p7_y_arr), pd.Series(p8_x_arr), pd.Series(p8_y_arr), pd.Series(p9_x_arr), pd.Series(p9_y_arr), pd.Series(p10_x_arr), pd.Series(p10_y_arr), pd.Series(p11_x_arr), pd.Series(p11_y_arr), pd.Series(p12_x_arr), pd.Series(p12_y_arr), pd.Series(p13_x_arr), pd.Series(p13_y_arr), pd.Series(p14_x_arr), pd.Series(p14_y_arr), pd.Series(p15_x_arr), pd.Series(p15_y_arr), pd.Series(p16_x_arr), pd.Series(p16_y_arr), pd.Series(label_arr)], axis=1, keys=header)

num_cols = grasp_dataset.select_dtypes(include=['float']).columns
df[num_cols] = grasp_dataset[num_cols].applymap(process_num)


In [36]:
pd.options.display.max_columns = None
print(grasp_dataset.head(3))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0             124.3             127.7                31.8                41.3   
1             135.2             119.5                31.6                37.0   
2             135.9             118.5                24.9                58.0   

   force_reading_1  force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0            231.4            231.7   0.0   0.0  0.477551  0.117114  0.807740   
1            246.4            248.1   0.0   0.0  0.480986  0.115637  0.813425   
2            238.1            239.6   0.0   0.0  0.484292  0.118618  0.821272   

       p3_y      p4_x      p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.321253  0.817246  0.671980  0.489952  0.950887  0.156755  0.709735   
1  0.327914  0.820319  0.674510  0.492725  0.937769  0.163668  0.708911   
2  0.319601  0.825090  0.668462  0.495218  0.957359  0.161158  0.706200   

       p7_x      p7_y      p8_x      p8_y      p9

In [37]:
grasp_dataset

,weight_reading_1,weight_reading_2,pressure_reading_1,pressure_reading_2,force_reading_1,force_reading_2,p1_x,p1_y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,p5_x,p5_y,p6_x,p6_y,p7_x,p7_y,p8_x,p8_y,p9_x,p9_y,p10_x,p10_y,p11_x,p11_y,p12_x,p12_y,p13_x,p13_y,p14_x,p14_y,p15_x,p15_y,p16_x,p16_y,label
0,124.3,127.7,31.8,41.3,231.4,231.7,0.0,0.0,0.477551,0.117114,0.807740,0.321253,0.817246,0.671980,0.489952,0.950887,0.156755,0.709735,0.460836,0.245851,0.452401,0.397521,0.446605,0.527526,0.455177,0.656923,0.470918,0.802496,0.510243,0.813511,0.521867,0.661269,0.527884,0.533579,0.518000,0.405263,0.500154,0.259419,test
1,135.2,119.5,31.6,37.0,246.4,248.1,0.0,0.0,0.480986,0.115637,0.813425,0.327914,0.820319,0.674510,0.492725,0.937769,0.163668,0.708911,0.460590,0.241665,0.451785,0.394455,0.446238,0.527171,0.454926,0.656818,0.470756,0.799687,0.514771,0.812474,0.525307,0.663050,0.530337,0.534389,0.519992,0.403325,0.504333,0.254939,test
2,135.9,118.5,24.9,58.0,238.1,239.6,0.0,0.0,0.484292,0.118618,0.821272,0.319601,0.825090,0.668462,0.495218,0.957359,0.161158,0.706200,0.466495,0.245109,0.457200,0.396219,0.450679,0.527006,0.458676,0.657007,0.474407,0.803137,0.517115,0.814139,0.528475,0.661528,0.534175,0.532647,0.524578,0.403266,0.507495,0.258334,test
3,61.0,62.4,9.1,11.2,76.7,77.5,0.0,0.0,0.480889,0.180898,0.811278,0.335709,0.815830,0.674720,0.498274,0.880368,0.167502,0.717638,0.461725,0.254720,0.462376,0.393396,0.460654,0.523790,0.468006,0.653712,0.477063,0.790413,0.515431,0.806264,0.513218,0.666471,0.513972,0.535805,0.506977,0.404484,0.500042,0.267848,test
4,61.0,62.5,7.0,8.7,99.8,100.2,0.0,0.0,0.479047,0.190793,0.804044,0.336701,0.807792,0.672772,0.494307,0.875705,0.170284,0.707052,0.462832,0.258752,0.463604,0.394565,0.462181,0.522681,0.469173,0.650008,0.477109,0.783352,0.510645,0.799123,0.508966,0.661279,0.510355,0.535350,0.502606,0.407658,0.494858,0.271824,test
5,59.2,64.6,23.8,19.4,115.1,115.7,0.0,0.0,0.478564,0.164959,0.805208,0.338048,0.811239,0.679951,0.495597,0.891869,0.169634,0.715459,0.462639,0.254477,0.462145,0.396408,0.459657,0.527389,0.466888,0.656500,0.477648,0.792351,0.512433,0.808386,0.513477,0.666569,0.515585,0.538960,0.506764,0.409625,0.497220,0.268260,test
6,75.3,93.5,74.1,32.9,227.9,229.7,0.0,0.0,0.481201,0.108376,0.808235,0.337804,0.814504,0.655281,0.496281,0.937036,0.159728,0.736666,0.461238,0.242242,0.448746,0.393515,0.442315,0.520183,0.452348,0.647508,0.471887,0.792774,0.511036,0.807008,0.521119,0.653723,0.527264,0.524508,0.517378,0.397331,0.499587,0.252330,test
7,82.7,84.3,56.9,44.4,223.7,224.4,0.0,0.0,0.480645,0.118757,0.803975,0.337321,0.814123,0.671781,0.492511,0.949444,0.162556,0.707043,0.462572,0.247967,0.452392,0.397702,0.446349,0.525119,0.455754,0.652598,0.472392,0.799741,0.509703,0.813355,0.519083,0.660937,0.523889,0.533278,0.513570,0.405557,0.497899,0.260842,test
8,78.8,88.8,58.5,73.1,215.6,216.3,0.0,0.0,0.477440,0.124447,0.798855,0.345739,0.809553,0.671376,0.492802,0.940315,0.162008,0.722471,0.456984,0.250695,0.446260,0.400095,0.441260,0.528684,0.451544,0.656426,0.469097,0.800814,0.508280,0.815290,0.515207,0.664562,0.519797,0.536377,0.509629,0.407486,0.494376,0.261804,test
9,98.3,90.3,60.1,64.5,226.2,227.3,0.0,0.0,0.481411,0.115495,0.808065,0.336491,0.814106,0.671670,0.491834,0.934821,0.164717,0.698799,0.464155,0.242550,0.455560,0.393945,0.449855,0.523615,0.458305,0.650507,0.473438,0.792484,0.509763,0.806298,0.519157,0.658167,0.524531,0.532241,0.514926,0.403871,0.500520,0.256272,test


In [ ]:
# grasp_dataset.to_csv('/home/dell/Desktop/xxxx/Uni/Kirigami_project/Keypoint_detection_notebooks/Grasp_dataset_3/' + 'validation_df.csv', index=False)

In [38]:
grasp_data = grasp_dataset.drop(columns=['label'], axis=1)
# grasp_data = grasp_dataset.drop(columns=['label', 'p1_x', 'p1_y', 'p2_x', 'p2_y', 'p3_x', 'p3_y', 'p4_x', 'p4_y', 'p5_x', 'p5_y', 'p6_x', 'p6_y', 'p7_x', 'p7_y', 'p8_x', 'p8_y', 'p9_x', 'p9_y', 'p10_x', 'p10_y', 'p11_x', 'p11_y', 'p12_x', 'p12_y', 'p13_x', 'p13_y', 'p14_x', 'p14_y', 'p15_x', 'p15_y', 'p16_x', 'p16_y'], axis=1)
grasp_data['weight_reading_1'] = grasp_data['weight_reading_1'] / 1000
grasp_data['weight_reading_2'] = grasp_data['weight_reading_2'] / 1000
grasp_data['pressure_reading_1'] = grasp_data['pressure_reading_1'] / 100
grasp_data['pressure_reading_2'] = grasp_data['pressure_reading_2'] / 100
grasp_data['force_reading_1'] = grasp_data['force_reading_1'] / 1000
grasp_data['force_reading_2'] = grasp_data['force_reading_2'] / 1000
grasp_label = grasp_dataset['label']

print(grasp_data.head(2))
print(grasp_label.head(2))

   weight_reading_1  weight_reading_2  pressure_reading_1  pressure_reading_2  \
0            0.1243            0.1277               0.318               0.413   
1            0.1352            0.1195               0.316               0.370   

   force_reading_1  force_reading_2  p1_x  p1_y      p2_x      p2_y      p3_x  \
0           0.2314           0.2317   0.0   0.0  0.477551  0.117114  0.807740   
1           0.2464           0.2481   0.0   0.0  0.480986  0.115637  0.813425   

       p3_y      p4_x     p4_y      p5_x      p5_y      p6_x      p6_y  \
0  0.321253  0.817246  0.67198  0.489952  0.950887  0.156755  0.709735   
1  0.327914  0.820319  0.67451  0.492725  0.937769  0.163668  0.708911   

       p7_x      p7_y      p8_x      p8_y      p9_x      p9_y     p10_x  \
0  0.460836  0.245851  0.452401  0.397521  0.446605  0.527526  0.455177   
1  0.460590  0.241665  0.451785  0.394455  0.446238  0.527171  0.454926   

      p10_y     p11_x     p11_y     p12_x     p12_y     p13_x  

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

data_train, data_test, label_train, label_test = train_test_split(grasp_data, grasp_label, test_size=0.2, random_state=np.random.randint(100))

# print(X_train.head(3))

# clf = KNeighborsClassifier(n_neighbors=7)

# knn.fit(X_train, y_train)

# y_pred = knn.predict(X_test)

# accuracy = knn.score(X_test, y_test)
# print('Accuracy:', accuracy)
# clf = RandomForestClassifier(n_estimators=1000, max_depth=50, random_state=np.random.randint(100))
clf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=np.random.randint(100), n_jobs = -1)
# clf = HistGradientBoostingClassifier(max_iter=200, random_state=np.random.randint(100))
# clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=1, random_state=0)
# clf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],voting='hard')

# num_folds = 10
# cv_method = KFold(n_splits=num_folds, shuffle=True, random_state=np.random.randint(100))
# cv_results = cross_val_score(clf, grasp_data, grasp_label, cv=cv_method, scoring='accuracy')

# print('Cross-validation results:', cv_results)
# print('Average accuracy:', cv_results.mean())

clf.fit(data_train, label_train)
label_pred = clf.predict(data_test)
cm = confusion_matrix(label_test, label_pred)

short_dataset_names = ['banana', 'chicken', 'egg', 'fig', 'greengrape', 'orange', 'pear', 'pistachio', 'redgrape', 'tomato', 'capsule', 'ibuprofen']


cm_df = pd.DataFrame(cm, index=short_dataset_names, columns=short_dataset_names)

print(cm_df)
print(cm)

accuracy = clf.score(data_test, label_test)
print("Accuracy:", accuracy)

            banana  chicken  egg  fig  greengrape  orange  pear  pistachio  \
banana          21        0    0    1           0       0     0          0   
chicken          0       15    0    0           0       0     3          0   
egg              0        0   20    2           0       0     0          0   
fig              0        0    0   19           0       0     0          0   
greengrape       0        0    0    0          21       0     0          0   
orange           0        0    0    0           0      15     0          0   
pear             0        2    0    0           0       0    15          0   
pistachio        0        0    0    0           0       0     0         26   
redgrape         0        0    0    0           0       0     0          0   
tomato           0        3    0    0           0       0     1          0   
capsule          0        1    0    0           0       0     1          0   
ibuprofen        0        0    0    0           0       0     0 

In [39]:
label_pred = clf.predict(grasp_data)
print(label_pred)

['banana' 'banana' 'tomato' 'capsule' 'capsule' 'capsule' 'tomato'
 'chicken' 'chicken' 'tomato' 'tomato' 'tomato' 'ibuprofen' 'pistachio'
 'pistachio' 'capsule' 'ibuprofen' 'ibuprofen' 'capsule' 'ibuprofen'
 'ibuprofen' 'orange' 'orange' 'orange' 'pear' 'pear' 'tomato']


In [ ]:
banana, capsule, chicken, egg, fig, greengrape, ibuprofen, orange, pear